In [2]:
%matplotlib inline
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pylab
import utils
import os
from PIL import Image

from torch import nn
from torch.autograd import Variable
from torchvision import models, transforms
import torch
import scipy.misc

In [3]:
# Load metadata and features.
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')

('track', 'tags')
('album', 'tags')
('artist', 'tags')
('track', 'genres')
('track', 'genres_all')


/Users/iaakhter/Documents/UBC/Winter22017/532L/DreamingInMusic/utils.py:218: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


## Scaling and Unscaling Spectrograms

In [11]:
# Find the min and max of spectrogram in all audio in directory
directory = "data/fma_small"
duration = 10 # duration of audio

minAmp = float("inf")
maxAmp = -float("inf")
for subdir, dirs, files in os.walk(directory):
    for file in files:
        filename = os.fsdecode(file)
        if filename.endswith(".mp3"): 
            #print (os.path.join(subdir, file))
            audioFilename = os.path.join(subdir, file)
            x, sr = librosa.load(audioFilename, sr=None, mono=True, duration=duration)
            #Convert audio to a complex valued spectrogram
            spectro = librosa.core.stft(x)

            #Separate out amplitude and phase from complex valued spectrogram
            mag, phase = librosa.core.magphase(spectro)
            #print ("mag", mag)
            #print ("phase",phase)

            #Get the decibal version from power spectrogram
            #This is the value that should be stored for training
            powerToDb = librosa.power_to_db(mag, ref=np.max)
            
            locMin = np.amin(powerToDb)
            locMax = np.amax(powerToDb)
            minAmp = min(minAmp, locMin)
            maxAmp = max(maxAmp, locMax)
            
print ("minAmp", minAmp)
print ("maxAmp", maxAmp)

minAmp -80.0
maxAmp 1.9073486e-06


In [4]:
minAmp = -80.0
maxAmp = 1.9073486e-06
def scaleSpectro(x, new_size):
    #print ("before scaling", x)
    x = (x - minAmp)/(maxAmp - minAmp)
    #print ("after scaling", x)
    y = scipy.misc.imresize(x, new_size, mode='L', interp='nearest')
    return y

In [5]:
def unscaleSpectro(x, new_size, minAmp, maxAmp):
    x = scipy.misc.imresize(x, new_size, mode='L', interp='nearest')
    x = x/255.0
    x = minAmp + x*(maxAmp - minAmp)
    return x

In [4]:

# return the first genreId of a track (a track can have multiple genres)
# rock genre id is 12, hiphop is 21 and pop is 10
def getGenreId(trackId):
    if len(tracks['track','genres'][trackId]) >= 1:
        return tracks['track','genres'][trackId][0]
    else:
        # if the track does not have a genre
        return None
    
# Define a global transformer to appropriately scale images and subsequently convert them to a Tensor.
array_size = 224
audioDirectory = "data/fma_small/"

def trackExists(trackIdNum):
    trackId = str(trackIdNum)
    while(len(trackId) < 6):
        trackId = "0" + trackId
    filename = trackId[0:3]+"/"+trackId
    audioFilename = audioDirectory + filename + ".mp3"
    if os.path.isfile(audioFilename):
        return True
    return False

# return processed spectrogram of a track
def loadSpectro(trackIdNum):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    trackId = str(trackIdNum)
    while(len(trackId) < 6):
        trackId = "0" + trackId
    filename = trackId[0:3]+"/"+trackId
    audioFilename = audioDirectory + filename + ".mp3"
    if trackExists(trackIdNum):
        x, sr = librosa.load(audioFilename, sr=None, mono=True)
        start, end = 0, 30
        ipd.Audio(data=x[start*sr:end*sr], rate=sr)
        #Convert audio to a complex valued spectrogram
        spectro = librosa.core.stft(x)

        #Separate out amplitude and phase from complex valued spectrogram
        mag, phase = librosa.core.magphase(spectro)
        #print ("mag", mag)
        #print ("phase",phase)

        #Get the decibal version from power spectrogram
        #This is the value that should be stored for training
        powerToDb = librosa.power_to_db(mag, ref=np.max)
        scaledSpectro = scaleSpectro(powerToDb, (array_size, array_size))
        spectroTensor = torch.from_numpy(scaledSpectro).type(torch.DoubleTensor)
        spectroVar = Variable(spectroTensor).unsqueeze(0)
        return spectroVar
    else:
        print("Audio does not exist")
        return None
genres
trackId = 2
print(genres['title'][getGenreId(trackId)])
loadSpectro(trackId)
    

Hip-Hop
before scaling [[-43.95143  -46.064735 -28.378609 ... -19.067362 -14.313644 -14.624919]
 [-44.87912  -42.24936  -28.243675 ... -14.731466 -12.89674  -18.953043]
 [-49.248756 -39.427902 -28.713446 ... -10.661632 -11.94172  -16.517147]
 ...
 [-80.       -64.72495  -61.05322  ... -43.127865 -40.401703 -40.65535 ]
 [-73.90991  -66.74268  -62.47602  ... -43.270184 -40.43517  -53.7668  ]
 [-72.58572  -72.90404  -64.10993  ... -43.26123  -40.452705 -40.642944]]
after scaling [[0.45060712 0.42419082 0.64526737 ... 0.761658   0.82107943 0.81718856]
 [0.439011   0.471883   0.64695406 ... 0.81585664 0.8387907  0.763087  ]
 [0.38439053 0.50715125 0.6410819  ... 0.8667296  0.8507285  0.79353565]
 ...
 [0.         0.19093809 0.23683476 ... 0.46090168 0.49497873 0.49180812]
 [0.0761261  0.16571645 0.21904974 ... 0.45912272 0.4945604  0.327915  ]
 [0.09267855 0.08869953 0.19862585 ... 0.45923463 0.4943412  0.4919632 ]]


/anaconda3/envs/myEnv2/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


Variable containing:
( 0 ,.,.) = 
  170  189  184  ...   216  214  216
  185  179  180  ...   222  223  218
  198  208  211  ...   200  196  225
      ...         ⋱        ...      
   41   59   55  ...    56   44   61
   61   53   66  ...    58   44   59
   57   47   65  ...    50   67   55
[torch.DoubleTensor of size 1x224x224]

## Check whether scaling and unscaling spectrogram causes loss in audio quality

In [10]:
#Original audio
x, sr = librosa.load("data/fma_small/000/000002.mp3", sr=None, mono=True, duration = 10)
start, end = 0, 30
ipd.Audio(data=x[start*sr:end*sr], rate=sr)

In [17]:
# Create spectrogram scale it and convert it to audio
spectro = librosa.core.stft(x)

#Separate out amplitude and phase from complex valued spectrogram
mag, phase = librosa.core.magphase(spectro)
#print ("mag", mag)
#print ("phase",phase)

#Get the decibal version from power spectrogram
#This is the value that should be stored for training
powerToDb = librosa.power_to_db(mag, ref=np.max)
scaledSpectro = scaleSpectro(powerToDb, (224,224))

unscaledSpectro = unscaleSpectro(scaledSpectro, powerToDb.shape, minAmp, maxAmp)

# Get the power spectrogram from decibal version
backToSpectro = (librosa.core.db_to_power(unscaledSpectro))
# Multiply power spectrogram with phase to get original complex spectrogram
backToSpectro = backToSpectro*phase

# Revert complex spectrogram to audio
n = len(x)
y_hat = librosa.istft(backToSpectro, length = n)
ipd.Audio(data=y_hat[start*sr:end*sr], rate=sr)

/anaconda3/envs/myEnv2/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys
/anaconda3/envs/myEnv2/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


## Example Training

In [6]:
#https://github.com/srviest/char-cnn-pytorch/blob/master/model_CharCNN2D.py
class SpectroCNN(nn.Module):
    def __init__(self, output_size):
        super(SpectroCNN, self).__init__()    
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1)),
            nn.ReLU()
        ).double()

        self.maxpool1 = nn.MaxPool2d(kernel_size=(4, 4), stride=(1, 1))
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(222, 222, kernel_size=(2, 2), stride=(1,1)),
            nn.ReLU()
        ).double()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(200, 200), stride=(4, 4))
        
        self.fc1 = nn.Sequential(
            nn.Linear(1800, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        ).double()
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        ).double()
        self.fc3 =nn.Linear(1024, output_size).double()
        self.softmax = nn.LogSoftmax()
            # nn.LogSoftmax()

        # self.inference_log_softmax = InferenceBatchLogSoftmax()

    def forward(self, x):
        debug=False
        x = x.unsqueeze(1)
        if debug:
            print('x.size()', x.size())

        x = self.conv1(x)
        if debug:
            print('x after conv1', x.size())

        x = x.transpose(1,3)
        if debug:
            print('x after transpose', x.size())

        x = self.maxpool1(x)
        if debug:
            print('x after maxpool1', x.size())

        x = self.conv2(x)
        if debug:
            print('x after conv2', x.size())

        x = x.transpose(1,3)
        if debug:
            print('x after transpose', x.size())

        x = self.maxpool2(x)
        if debug:
            print('x after maxpool2', x.size())

        x = x.view(x.size(0), -1)
        if debug:
            print('Collapse x:, ', x.size())

        x = self.fc1(x).type(torch.DoubleTensor)
        if debug:
            print('FC1: ', x.size())

        x = self.fc2(x)
        if debug:
            print('FC2: ', x.size())

        x = self.fc3(x)
        if debug:
            print('x: ', x.size())

        x = self.softmax(x)
        # x = self.inference_log_softmax(x)

        return x



In [7]:
train_ids = [2,5,10,140,141,148,182,190,193,194]
tempCat = {'Pop':0, 'Rock':1, 'Hip-Hop':2}

def create_training(start, end):
    training_input = [loadSpectro(train_id).squeeze() for train_id in train_ids[start:end]
                      if trackExists(train_id)]

    # The output data is prepared by representing each output as a binary vector of categories
    training_output = []
    for i in range(start,min(len(train_ids),end)):
        if not(trackExists(train_ids[i])):
            continue
        genre = genres['title'][getGenreId(trackId)]
        training_vector = np.zeros(len(tempCat))
        training_vector[tempCat[genre]] = 1
        training_output.append(training_vector)
    training_output = Variable(torch.FloatTensor(training_output))
    
    return training_input, training_output

def train(model, learning_rate=0.0001, batch_size=1, epochs=1):
    """
    Training function which takes as input a model, a learning rate and a batch size.
  
    After completing a full pass over the data, the function exists, and the input model will be trained.
    """
    # Define the criterion and optimizer.
    criterion = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Keep track of the losses, for the purposes of plotting.
    train_losses = []

    # Determine number of minibatches
    num_iter = epochs * len(train_ids)//batch_size 
    for i in range(num_iter):
        print("Starting iteration: ", i)
        
        start_idx = i * batch_size % len(train_ids)
        
        training_input, training_output = create_training(start_idx, start_idx + batch_size)

        # Retrieve the next batch of training data.
        x = torch.stack(training_input)
        y = training_output.type(torch.DoubleTensor)

        # Forward pass
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y)

        # Zero gradients, perform backwards pass and update model weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        
        del x,y,y_pred

        if i % 1 == 0:
            train_losses.append(loss.data[0])
            print(i, train_losses[-1])
        
    return train_losses 

# Finally train the model
numClasses = 3
model = SpectroCNN(numClasses)
model.train()
train(model)

Starting iteration:  0


/anaconda3/envs/myEnv2/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 5.423885960115509
Starting iteration:  1
1 1.8480728067984742
Starting iteration:  2
2 0.23472917934248896
Starting iteration:  3
3 0.23104908751567466
Starting iteration:  4
4 0.2310490601853169
Starting iteration:  5
5 0.23104906018531735
Starting iteration:  6
6 0.23104906018531504
Starting iteration:  7
7 0.23104908994478238
Starting iteration:  8
8 0.23104906018531504
Starting iteration:  9
9 0.23104906018531504


[5.423885960115509,
 1.8480728067984742,
 0.23472917934248896,
 0.23104908751567466,
 0.2310490601853169,
 0.23104906018531735,
 0.23104906018531504,
 0.23104908994478238,
 0.23104906018531504,
 0.23104906018531504]